# Preprocessing of Temperature and Breizhcrops datasets
In the end, we would like to have a dataset where satellites data are associated with the temperature cumulative sum. 

We need to create a h5py file, like the ones provided by breizhcrops. The values have a difference spacing, according to the temperature cumulative sum. 

In [2]:
import sys
sys.path.append("..")
import os 
import glob
import netCDF4
from data import BreizhCrops
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
from utils.temperature_data.load import get_time_data, get_temperature_data, get_lon_data, get_lat_data, load_nc_files